In [3]:
import yfinance as yf  # Data
import pandas as pd  # DataFrame
import plotly.graph_objects as go  # Graph
import time

In [ ]:
# Fix: เก็บข้อมูล asset ใน class
class Asset:
    def __init__(self, name):
        self.name = name
        self.prices = []
        self.volumes = []

    def add(self, price, volume):
        self.prices.append(price)
        self.volumes.append(volume)

    def fifo(self):
        if self.prices and self.volumes:
            self.prices.pop(0)
            self.volumes.pop(0)
class Portfolio:
    def __init__(self, cash, port=None):
        if port is None:
            port = {}
        self.cash = cash
        self.portfolio = port
        self.total_value = 0

    def add_asset(self, symbol):
        self.portfolio[symbol] = Asset(symbol)

    def buying(self, hist, amount, asset):  # amount in dollars
        if amount > self.cash:
            print("Not enough cash to buy Bitcoin")
            return self.cash
        if asset not in self.portfolio:
            self.portfolio[asset] = Asset(asset)
        self.portfolio[asset].add(hist, amount / hist)
        self.cash -= amount
        return self.cash
        # update cash

    def selling(self, amount, asset):
        total_vol = self.get_total_volume(asset)
        if amount > total_vol:
            print(f"Not enough {asset} to sell")
            return self.cash
        if asset not in self.portfolio:
            print(f"No {asset} to sell")
            return self.cash

        volumes = self.portfolio[asset].volumes
        prices = self.portfolio[asset].prices

        while amount > 0 and volumes:
            if amount >= volumes[0]:
                total_sell = amount * prices[0]
                amount -= volumes[0]
                self.cash += total_sell
                self.portfolio[asset].fifo()
            elif amount < volumes[0]:
                volumes[0] -= amount
                self.cash += amount * prices[0]
                amount = 0
        if not volumes:
            del self.portfolio[asset]

        return self.cash

    def get_total_volume(self, asset):
        total = sum(self.portfolio[asset].volumes)
        return total

    def get_assets(self):
        return self.portfolio

    def total_asset_value(self, hist, asset):
        if asset not in self.portfolio:
            print(f"{asset} not in portfolio")
            return 0
        total = hist["Close"].iloc[-1] * sum(self.portfolio[asset].volumes)
        self.total_value = total
        return total

    def __str__(self):
        return f"Total port's value: {self.cash + self.total_value} USD"


def HISTstockPrice(stock, period, interval):

    ticker = yf.Ticker(stock)
    data = ticker.history(period=period, interval=interval)  # 1-minute data for today
    df = pd.DataFrame(data)

    df["diff"] = df["Close"] - df["Open"]
    df["color"] = df["diff"].apply(lambda x: "green" if x >= 0 else "red")

    df.to_csv(f"{stock}.csv")
    return df

# 50 50 criteria

In [50]:
#####################
asset_name = "BTC-USD"
#####################
hist = HISTstockPrice(asset_name, "1y", "1d")

# implement rebalancing 50:50
Cash = 1000000  # dollars
port2 = Portfolio(Cash)


def simulation5050(hist, port, asset_name):
    buyCount = 0
    sellcount = 0
    for i in range(hist.shape[0]):

        total_btc_value = port.total_asset_value(hist, asset_name)
        price = hist["Close"].iloc[i]
        # Fix: vvvvvvvvvvvvvvvvvv ทำเป็น  method: port.get_total_volume()
        total_vol = (
            port.get_total_volume(asset_name)
            if asset_name in port.portfolio
            else 0
        )
    
        if asset_name not in port.portfolio:
            port.buying(price, 0.5 * port.cash, asset_name)
            buyCount += 1
            print(f"Remaining cash: {port.cash:.2f} USD, BTCval : {total_btc_value}")
            print(f"Updated portfolio: {port.portfolio[asset_name].volumes}")
        else:
            if (price * total_vol) / ((price * total_vol) + port.cash) > 0.5:
                dollars_amount = ((price * total_vol) - port.cash) / 2
                btc_amount = dollars_amount / price
                port.selling(
                    btc_amount, 
                    asset_name,
                )
                sellcount += 1
            elif (price * total_vol) / ((price * total_vol) + port.cash) < 0.5:
                dollars_amount = abs(((price * total_vol) - port.cash) / 2)
                port.buying(price, dollars_amount, asset_name)
                buyCount += 1
            if port.cash < 10000:
                print("Cash is too low, stopping trading.")
                break
            print(f"Remaining cash: {port.cash:.2f} USD, BTCval : {total_btc_value}")
            print(f"Updated portfolio: {port.portfolio[asset_name].volumes}")
    
    total_btc_value = port.total_asset_value(hist, asset_name)
    total_vol = port.get_total_volume
    
    # Fix: vvvvvvvvvvvvvvvvvvvvvvvvv ใช้ method __str__
    print(total_btc_value, port, port.cash, total_vol)
    print(f"Total buy count: {buyCount}, Total sell count: {sellcount}")


simulation5050(hist, port2, asset_name)

BTC-USD not in portfolio
Remaining cash: 500000.00 USD, BTCval : 0
Updated portfolio: [np.float64(8.635658401149676)]
Remaining cash: 505624.03 USD, BTCval : 1016185.9899530861
Updated portfolio: [np.float64(8.538524058845224)]
Remaining cash: 512012.35 USD, BTCval : 1004755.8762075088
Updated portfolio: [np.float64(8.428189354661336)]
Remaining cash: 527509.40 USD, BTCval : 991772.4329784021
Updated portfolio: [np.float64(8.160534821714332)]
Remaining cash: 529162.92 USD, BTCval : 960276.6542092961
Updated portfolio: [np.float64(8.131976384641304)]
Remaining cash: 525287.71 USD, BTCval : 956916.0901039923
Updated portfolio: [np.float64(8.131976384641304), np.float64(0.060437888915449066)]
Remaining cash: 524694.88 USD, BTCval : 964028.0129158121
Updated portfolio: [np.float64(8.131976384641304), np.float64(0.060437888915449066), np.float64(0.009266723400531331)]
Remaining cash: 534433.24 USD, BTCval : 965118.4583752035
Updated portfolio: [np.float64(7.9637820845608545), np.float64(0.0

# Threshold Criteria

In [51]:
#####################
asset_name = "BTC-USD"
#####################
hist = HISTstockPrice(asset_name, "1y", "1d")

# implement rebalancing 50:50
Cash = 1000000  # dollars
port2 = Portfolio(Cash)


def simulationThreshold(hist, port, asset_name):
    buyCount = 0
    sellcount = 0
    for i in range(hist.shape[0]):

        total_btc_value = port.total_asset_value(hist, asset_name)
        price = hist["Close"].iloc[i]
        # Fix: vvvvvvvvvvvvvvvvvv ทำเป็น  method: port.get_total_volume()
        total_vol = (
            port.get_total_volume(asset_name) if asset_name in port.portfolio else 0
        )

        if asset_name not in port.portfolio:
            port.buying(price, 0.5 * port.cash, asset_name)
            buyCount += 1
            print(f"Remaining cash: {port.cash:.2f} USD, BTCval : {total_btc_value}")
            print(f"Updated portfolio: {port.portfolio[asset_name].volumes}")
        else:
            if ((price * total_vol) / (port.cash)) - 1 > 0.1:
                dollars_amount = ((price * total_vol) - port.cash) / 2
                btc_amount = dollars_amount / price
                port.selling(
                    btc_amount,
                    asset_name,
                )
                sellcount += 1
            elif ((price * total_vol) / (port.cash)) - 1 < -0.05:
                dollars_amount = abs(((price * total_vol) - port.cash) / 2)
                port.buying(price, dollars_amount, asset_name)
                buyCount += 1
            if port.cash < 10000:
                print("Cash is too low, stopping trading.")
                break
            print(f"Remaining cash: {port.cash:.2f} USD, BTCval : {total_btc_value}")
            print(f"Updated portfolio: {port.portfolio[asset_name].volumes}")

    total_btc_value = port.total_asset_value(hist, asset_name)
    total_vol = port.get_total_volume

    # Fix: vvvvvvvvvvvvvvvvvvvvvvvvv ใช้ method __str__
    print(total_btc_value, port, port.cash, total_vol)
    print(f"Total buy count: {buyCount}, Total sell count: {sellcount}")


simulationThreshold(hist, port2, asset_name)

BTC-USD not in portfolio
Remaining cash: 500000.00 USD, BTCval : 0
Updated portfolio: [np.float64(8.635658401149676)]
Remaining cash: 500000.00 USD, BTCval : 1016578.0353512821
Updated portfolio: [np.float64(8.635658401149676)]
Remaining cash: 500000.00 USD, BTCval : 1016578.0353512821
Updated portfolio: [np.float64(8.635658401149676)]
Remaining cash: 526864.00 USD, BTCval : 1016578.0353512821
Updated portfolio: [np.float64(8.171681719299293)]
Remaining cash: 526864.00 USD, BTCval : 961959.327457339
Updated portfolio: [np.float64(8.171681719299293)]
Remaining cash: 526864.00 USD, BTCval : 961959.327457339
Updated portfolio: [np.float64(8.171681719299293)]
Remaining cash: 526864.00 USD, BTCval : 961959.327457339
Updated portfolio: [np.float64(8.171681719299293)]
Remaining cash: 526864.00 USD, BTCval : 961959.327457339
Updated portfolio: [np.float64(8.171681719299293)]
Remaining cash: 526864.00 USD, BTCval : 961959.327457339
Updated portfolio: [np.float64(8.171681719299293)]
Remaining ca